<a href="https://colab.research.google.com/github/curyumafu/sakana/blob/main/%E7%94%BB%E5%83%8F%E5%88%86%E9%A1%9E_transfer(InceptionResNetV2)2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit==1.20.0 --quiet
!pip install pyngrok==4.1.1 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 71.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
import streamlit as st
from pyngrok import ngrok

In [3]:
%%writefile app.py
from google.colab import drive, output
from google.colab.patches import cv2_imshow
from IPython.display import display, Javascript
from IPython.display import JSON as IPython_JSON
from PIL import Image, ImageFont, ImageDraw
import pickle
import cv2
import numpy as np
from io import BytesIO
import base64
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input, decode_predictions
from keras.preprocessing import image
import tensorflow as tf



drive.mount('/content/drive')
%cd /content/drive/MyDrive/実験III_AI演習（ImageNet）2023



# クラスラベルの読み込み
classes = pickle.load(open('classes.sav', 'rb'))



# クラスラベルをリストとして表示
for i, class_label in enumerate(classes):
    print(f"Class {i}: {class_label}")



def img_add_msg(img, message):
    """画像にメッセージを追加する関数"""
    font_path = 'ipaexg.ttf'
    font_size = 20
    font = ImageFont.truetype(font_path, font_size)  # フォントの定義
    draw = ImageDraw.Draw(img)  # 描画用のDraw関数
    draw.text((0, 30), message, font=font, fill=(255, 255, 255, 0))  # テキストを描画
    return img



# モデルの読み込み (SavedModel 形式)
model = tf.keras.models.load_model('./cnn_saved_model')



# コメントを返す関数（クラスごとに変更が必要です）
def get_comment_for_class(class_label):
    # ここで各クラスに対応したコメントを返すように実装してください
    # 例: クラス 'cat' に対応したコメント
    if class_label == 'アカザ':
        return "この魚はアカザという魚です。\n胸びれと背びれの部分に\n毒がある可能性があるため\n気をつけてください"
    # 例: クラス 'dog' に対応したコメント
    elif class_label == 'アユ':
        return "この魚はアユという魚です。\n毒は特にありません"
    elif class_label == 'ウナギ':
        return "この魚はウナギという魚です。\n体に毒はありませんが\n新鮮な血には毒があるため気を付けてください"
    elif class_label == 'カジカ':
        return "この魚はカジカという魚です。\n毒は特にありません"
    elif class_label == 'コイ':
        return "この魚はコイという魚です。\nコイの胆嚢にはコイ毒が含まれている\n可能性があるため調理する際は気をつけてください"
    elif class_label == 'サケ':
        return "この魚はサケという魚です。\n毒は特にありません"
    elif class_label == 'サンフィッシュ':
        return "この魚はサンフィッシュという魚です。\n毒は特にありません"
    elif class_label == 'ドジョウ':
        return "この魚はドジョウという魚です。\n毒は特にありません"
    elif class_label == 'ナマズ':
        return ""
    elif class_label == 'ハゼ':
        return "これは9で釣れます"
    elif class_label == '背景':
        return "これは背景です"
    else:
        return "No specific comment for this class yet."



def run(img_str):
    """画像処理と予測を行う関数"""
    # 画像のデコード
    decimg = base64.b64decode(img_str.split(',')[1], validate=True)
    decimg = Image.open(BytesIO(decimg))
    decimg = np.array(decimg, dtype=np.uint8)
    decimg = cv2.cvtColor(decimg, cv2.COLOR_BGR2RGB)



    # 画像のトリミングと前処理
    trim_frame = decimg[:299, :299]
    x = image.img_to_array(trim_frame)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)



    # 予測
    preds = model.predict(x, verbose=0)
    prelabel = np.argmax(preds, axis=1)[0]
    class_label = classes[prelabel]
    confidence = preds[0][prelabel] * 100
    message = f"{class_label} {confidence:.2f}%"



    # クラスに対応したコメントを取得
    comment = get_comment_for_class(class_label)



    # 画像にメッセージとコメントを追加
    img = Image.fromarray(trim_frame)
    img = img_add_msg(img, f"{message}\n{comment}")
    img = np.array(img)



    # 画像をエンコードして送信
    _, encimg = cv2.imencode(".jpg", img, [int(cv2.IMWRITE_JPEG_QUALITY), 80])
    img_str = encimg.tobytes()
    img_str = "data:image/jpeg;base64," + base64.b64encode(img_str).decode('utf-8')
    return IPython_JSON({'img_str': img_str})



# コールバックの登録
output.register_callback('notebook.run', run)



from google.colab.output import eval_js



def use_cam(quality=0.8):
    """ウェブカメラを使用して画像を取得し、処理を行う関数"""
    js = Javascript('''
    async function useCam(quality) {
      const div = document.createElement('div');
      document.body.appendChild(div);
      //video element
      const video = document.createElement('video');
      video.style.display = 'None';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      //canvas for display. frame rate is depending on display size and jpeg quality.
      display_size = 299
      const src_canvas = document.createElement('canvas');
      src_canvas.width  = display_size;
      src_canvas.height = display_size;;
      const src_canvasCtx = src_canvas.getContext('2d');
      src_canvasCtx.translate(src_canvas.width, 0);
      src_canvasCtx.scale(-1, 1);
      div.appendChild(src_canvas);

      const dst_canvas = document.createElement('canvas');
      dst_canvas.width  = src_canvas.width;
      dst_canvas.height = src_canvas.height;
      const dst_canvasCtx = dst_canvas.getContext('2d');
      div.appendChild(dst_canvas);

      //exit button
      const btn_div = document.createElement('div');
      document.body.appendChild(btn_div);
      const exit_btn = document.createElement('button');
      exit_btn.textContent = 'Exit';
      var exit_flg = true
      exit_btn.onclick = function() {exit_flg = false};
      btn_div.appendChild(exit_btn);

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      var send_num = 0
      // loop
      _canvasUpdate();
      async function _canvasUpdate() {
            src_canvasCtx.drawImage(video, 0, 0, video.videoWidth, video.videoHeight, 0, 0, src_canvas.width, src_canvas.height);
            if (send_num<1){
                send_num += 1
                const img = src_canvas.toDataURL('image/jpeg', quality);
                const result = google.colab.kernel.invokeFunction('notebook.run', [img], {});
                result.then(function(value) {
                    parse = JSON.parse(JSON.stringify(value))["data"]
                    parse = JSON.parse(JSON.stringify(parse))["application/json"]
                    parse = JSON.parse(JSON.stringify(parse))["img_str"]
                    var image = new Image()
                    image.src = parse;
                    image.onload = function(){dst_canvasCtx.drawImage(image, 0, 0)}
                    send_num -= 1
                })
            }
            if (exit_flg){
                requestAnimationFrame(_canvasUpdate);
            }else{
                stream.getVideoTracks()[0].stop();
            }
      };
    }
    ''')
    display(js)
    eval_js('useCam({})'.format(quality))

# ウェブカメラの使用を開始
use_cam()

Writing app.py


In [4]:
!ngrok authtoken 2a1EBSIaJWXov1REo9hZuErXTIY_352Vf6KGRyvZtMFD7Uxrn

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [5]:
!streamlit run app.py &>/dev/null&  # 「&>/dev/null&」により、出力を非表示にしてバックグランドジョブとして実行

In [7]:
ngrok.kill()  # プロセスの修了
url = ngrok.connect(port="8501")  # 接続

INFO:pyngrok.process:ngrok process starting: 1506
2024-01-15 04:24:19.043 INFO    pyngrok.process: ngrok process starting: 1506
INFO:pyngrok.process:t=2024-01-15T04:24:19+0000 lvl=info msg="no configuration paths supplied"

2024-01-15 04:24:19.070 INFO    pyngrok.process: t=2024-01-15T04:24:19+0000 lvl=info msg="no configuration paths supplied"

INFO:pyngrok.process:t=2024-01-15T04:24:19+0000 lvl=info msg="using configuration at default config path" path=/root/.ngrok2/ngrok.yml

2024-01-15 04:24:19.074 INFO    pyngrok.process: t=2024-01-15T04:24:19+0000 lvl=info msg="using configuration at default config path" path=/root/.ngrok2/ngrok.yml

INFO:pyngrok.process:t=2024-01-15T04:24:19+0000 lvl=info msg="open config file" path=/root/.ngrok2/ngrok.yml err=nil

2024-01-15 04:24:19.082 INFO    pyngrok.process: t=2024-01-15T04:24:19+0000 lvl=info msg="open config file" path=/root/.ngrok2/ngrok.yml err=nil

INFO:pyngrok.process:t=2024-01-15T04:24:19+0000 lvl=info msg="starting web service" obj=

In [8]:
print(url)

http://a8f7-34-143-168-249.ngrok-free.app


In [ ]:
from jupyter_dash import JupyterDash
import dash_bootstrap_components as dbc

class JupyterDashBootStrap(JupyterDash):
  def __init__(self, _title, _port):
    super().__init__(__name__, external_stylesheets=self.__external_stylesheets())
    if _title is not None:
      self.title = _title
    self.__setlayout()
    self.__run(_port)

  def __setlayout(self):
      self.layout = self.buildbaselayout()
      self.registcalback(self)

  def buildbaselayout(self):
      return

  def registcalback(self, app):
      return

  def __run(self, _port):
    if _port is None:
      self.run_server(mode="external")
    else:
      self.run_server(mode="external", port=_port )

  def __external_stylesheets(self):
    return [dbc.themes.BOOTSTRAP]

ImportError: ignored